# Carnival Demo

In [1]:
import serial
from LatLongMap import latitude_values, longitude_values
from gpt import GPTWaterLevelModel
import torch
import numpy as np
import pandas as pd

In [2]:
device = 'cpu'
start_row = 0

In [3]:
def getIndexForXposition(x, start_value = 45.00, end_value = 3.00, step = 0.01, num_indices = 275):
    normalised_position = (start_value - x)/(start_value - end_value)
    index = normalised_position * (num_indices - 1)
    index = abs(int(index))
    #print(index)
    return index

In [4]:
def createDataForModel(tensor, stack_vector):
    tensor = tensor.unsqueeze(0).unsqueeze(0)
    # If stack_vector is initially of size [1, 0, 0], initialize it with the new tensors
    if stack_vector.size() == torch.Size([1, 0, 0]):
        stack_vector = tensor
    else:
        # Otherwise, concatenate the new tensors along the second dimension
        #new_tensor = torch.stack(tensor).unsqueeze(0).unsqueeze(0)
        stack_vector = torch.cat((stack_vector, tensor), dim=1)    
    return stack_vector

In [5]:
def getNextCoordinates(stack_vector, count=4):
    nextLatLong = torch.empty((1,0, 0))
    #Get the lat and long from the last element of the vector
    last_entry_lat_long = stack_vector[0, -1, 0:2]
    #print(last_entry_lat_long)
    #Find the index of current latitude
    lat = last_entry_lat_long[0]   
    ##print(lat)
    index = latitude_values.index(lat)
    #print(index)
    for i in range(count):
        index = index+1
        tensor = torch.tensor([latitude_values[index], longitude_values[index]], dtype=torch.float32, device=device)
        tensor = tensor.unsqueeze(0).unsqueeze(0)
        if nextLatLong.size() == torch.Size([1, 0, 0]):
            nextLatLong = tensor
        else:
            nextLatLong = torch.cat((nextLatLong, tensor),dim = 1)
    return nextLatLong   

In [ ]:
def sendInfoToHMI(CurrentWaterLevels, predicted_water_levels):
    global start_row
    CurrentWaterLevels = CurrentWaterLevels[:, -1: , :].squeeze()
    array1 = CurrentWaterLevels.numpy().flatten()
    #print(array1.size)
    predicted_water_levels = predicted_water_levels.squeeze()
    array2 = predicted_water_levels.numpy().flatten()
    #print(array2.size)    
    combined_array = np.hstack((array1, array2))
    #print(combined_array.size)
    # df_combined = pd.DataFrame([combined_array], columns = [f'Column{i+1}' for i in range(combined_array.size)])
    # df_combined.to_excel('ToHMI.xlsx', sheet_name='tohmi',index=False, startrow=start_row)
    # start_row = start_row + 1
    values_list = combined_array.flatten().astype(str).tolist()
    with open('ToHMI.txt','a') as file:
        line = '\t'.join(values_list)
        file.write(line + '\n')
  

In [ ]:
def getWaterLevelPredictions(CurrentWaterLevels, model):
    next_coordinates = getNextCoordinates(CurrentWaterLevels)
    #print(next_coordinates)
    #print(CurrentWaterLevels)
    CurrentWaterLevels = CurrentWaterLevels[:, -8: , :]
    predicted_water_levels = model.generate(CurrentWaterLevels, 4, next_coordinates) 
    print(f'Current 8(upto) water levels = {[round(value, 4) for value in CurrentWaterLevels[:,:,-1].tolist()[0]]}' )
    print(f'Predicted next 4 water levels = {[round(value,4) for value in predicted_water_levels[:,8:,-1].tolist()[0]]}' )
    sendInfoToHMI(CurrentWaterLevels, predicted_water_levels[:,8:,:])

In [53]:
def read_data(ser):    
    global prev_depth
    global prev_loc
    
    if ser.in_waiting > 0:
        raw_data = ser.readline().strip()
        raw_data.decode('latin1')
        #buf = ser.readline().decode('utf-8').rstrip()
        buf = str(raw_data).rstrip("\n")
        #print(buf)
        start = buf.find('[')
        #print(start)
        buf = buf[start+1:]
        end = buf.find(']')
        #print(end)
        msg_parts = buf[:end].split(";") # buf now has depth, x-axis
        #print(msg_parts)
        curWaterDepth = float(msg_parts[0])
        x_position = float(msg_parts[1])
        if (x_position >= 8.0) and (x_position <= 44.25) and (x_position < prev_loc) and (curWaterDepth < prev_depth):
            prev_loc = x_position
            prev_depth = curWaterDepth
            index = getIndexForXposition(float(msg_parts[1]))
            #print(index)
            lat = latitude_values[index]
            long = longitude_values[index]
            tensor = torch.tensor([lat, long, curWaterDepth], dtype=torch.float32, device=device)      
            return tensor



In [ ]:
running = True
ser = serial.Serial("COM9", 9600)
saved_model = GPTWaterLevelModel(800)
saved_model.load_state_dict(torch.load('saved_model_new.pth', map_location=torch.device('cpu')))
LatLongDepthTensor = torch.empty((1, 0, 0))
while running:
    tensor = read_data(ser)    
    if tensor is not None:
        #print(tensor)
        LatLongDepthTensor = createDataForModel(tensor, LatLongDepthTensor)
        getWaterLevelPredictions(LatLongDepthTensor, saved_model)

In [ ]:
ser.close() 
prev_depth = 100
prev_loc = 1000

In [ ]:
# df = pd.read_excel('dataset5.xlsx')
# water_depth = df.iloc[:, 0]
# x_pos = df.iloc[:, 1]
# start_value = 45.00
# end_value = 3.00
# step = 0.01
# num_indices = 275
# normalised_position = (start_value - x_pos.values)/(start_value - end_value)
# index = normalised_position * (num_indices - 1)
# indices = [abs(int(x)) for x in index]
# lat = [latitude_values[x] for x in indices]
# long = [longitude_values[x] for x in indices]
# df2 = pd.DataFrame({'lat':lat, 'long':long, 'depth':water_depth})
# df2.to_excel('ActLatLongDepth1.xlsx', index=False, header=False)

In [58]:
# running = True
# ser = serial.Serial("COM9", 9600)
# saved_model = GPTWaterLevelModel(800)
# saved_model.load_state_dict(torch.load('saved_model_new.pth', map_location=torch.device('cpu')))
# LatLongDepthTensor = torch.empty((1, 0, 0))
# prev_depth = 100
# prev_loc = 1000
# while running:
#     tensor = read_data(ser)    
#     if tensor is not None:
#         #print(tensor)
#         LatLongDepthTensor = createDataForModel(tensor, LatLongDepthTensor)

C:\Users\srtoleti\AppData\Local\Temp\ipykernel_33388\3341988633.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_model.load_state_dict(torch.load('saved_model_new.p

KeyboardInterrupt: 

In [59]:
# ser.close() 
# prev_depth = 100
# prev_loc = 1000
# LatLongDepthTensor = LatLongDepthTensor.squeeze(0)
# print(LatLongDepthTensor.size())
# lat = LatLongDepthTensor[:,0].tolist()
# long = LatLongDepthTensor[:,1].tolist()
# water_depth = LatLongDepthTensor[:,2].tolist()
# df2 = pd.DataFrame({'lat':lat, 'long':long, 'depth':water_depth})
# df2.to_excel('TrainLatLongDepth16.xlsx', index=False, header=False)

torch.Size([5, 3])


In [ ]:
LatLongDepthTensor